In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.utils.data
import pickle
import os

In [2]:
def eval(model, eval_loader):
    model.eval()
    total_loss = 0
    correct = 0

    for batch_fix, (data, target) in enumerate(eval_loader):
        prediction = model(data)
        loss = cost_function(prediction, target)
        total_loss += loss.data[0]*len(data)
        pred_classes = prediction.data.max(1,keepdim=True)[1]
        correct += pred_classes.eq(target.data.view_as(pred_classes)).sum().double()

    # compute mean loss
    mean_loss = total_loss/len(eval_loader.dataset)
    acc = correct/len(eval_loader.dataset)
    print('Eval:  Avg_Loss: {:.5f}   Acc: {}/{} ({:.3f}%)'.format(
        mean_loss, correct, len(eval_loader.dataset),
        100. * acc))
    return mean_loss, acc

def cost_function(prediction, target):
    loss = F.cross_entropy(prediction, target)
    return loss


In [3]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 20)
        self.fc2 = nn.Linear(20, 40)
        self.fc3 = nn.Linear(40, 40)
        self.fc4 = nn.Linear(40, 80)
        self.fc5 = nn.Linear(80, 40)
        self.fc6 = nn.Linear(40, 20)
        self.fc7 = nn.Linear(20, 11)         # Final output: predicing # of possible accidents

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc3(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc4(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc5(out))
        out = F.dropout(out, p=0.5)
        out = F.relu(self.fc6(out))
        out = F.dropout(out, p=0.5)
        out = self.fc7(out)
        return out

In [4]:
neural_net = Net()

In [5]:
df = pd.read_csv("data.csv")

train, validate, test = np.split(df.sample(frac=1, random_state=134),
                                 [int(.6*len(df)), int(.8*len(df))])

x_train = train.drop(['Count'], axis =1).values
y_train = train['Count'].values

x_val = train.drop(['Count'], axis=1).values
y_val = train['Count'].values

x_test = test.drop(['Count'], axis=1).values
y_test = test['Count'].values


train_data = torch.utils.data.TensorDataset(torch.from_numpy(x_train).float(),
                                            torch.from_numpy(y_train).long())
val_data = torch.utils.data.TensorDataset(torch.from_numpy(x_val).float(),
                                          torch.from_numpy(y_val).long())
test_data = torch.utils.data.TensorDataset(torch.from_numpy(x_test).float(),
                                           torch.from_numpy(y_test).long())


train_batch_size = 32
eval_batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size = train_batch_size, shuffle =True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = eval_batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = eval_batch_size, shuffle=False)


In [6]:
test_loss, test_acc = eval(neural_net, test_loader)

Eval:  Avg_Loss: 2.79160   Acc: 46.0/749 (6.142%)


/Users/ShereenElaidi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  if __name__ == '__main__':


In [7]:

# neural_net.load_state_dict(torch.load('Model_optimal.pth'))
a = torch.load('Model_optimal.pth')


In [8]:
neural_net

Net(
  (fc1): Linear(in_features=5, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=80, bias=True)
  (fc5): Linear(in_features=80, out_features=40, bias=True)
  (fc6): Linear(in_features=40, out_features=20, bias=True)
  (fc7): Linear(in_features=20, out_features=11, bias=True)
)